[MAL API Documentation](https://myanimelist.net/apiconfig/references/api/v2#section/Versioning)

In [ ]:
import json
import numpy as np
import os
import pandas as pd
from dotenv import find_dotenv, load_dotenv
from helper.loadMALJSONIntoDF import loadMALJSONIntoDF
from helper.misc import canUpdateData
from helper.download_data import myAnimeData, hofAnimeData
from dateutil.relativedelta import relativedelta

In [ ]:
dotenv_file = find_dotenv()
load_dotenv(dotenv_file)

print('Checking HOF Anime Data Import date...')
if canUpdateData(dotenv_file, 'nextHOFImportDate', '%Y-%m-%d', 12):
    themes_tags = ['aliens', 'androids', 'angels', 'angst', 'anthropomorphism', 'comingofage', 'conspiracy', 'deities', 'delinquents', 'detectives', 'disasters', 'dragons', 'elves', 'fairies', 'friendship', 'ghosts', 'gore', 'gunfights', 'historicalperiod', 'lovetriangles', 'maids', 'mythology', 'ninjas', 'parentalabandonment', 'powersuits', 'robots', 'slapstick', 'stereotypes', 'swordplay', 'tragedy', 'awardnotv', 'awardtv', 'timetravel', 'battleroyale', 'isekai']

print()

print('Checking MAL Anime Data Import date...')
if canUpdateData(dotenv_file, 'nextMALImportDate_Anime', '%Y-%m-%d', 2):
    clientId = os.getenv('X-MAL-CLIENT-ID')
    animeList = os.getenv('myAnimeList')
    fields = ['rating, media_type, genres, studios, list_status']
    filePath = './data/MAL Anime Data/'
    myAnimeData(clientId, animeList, fields, filePath)

In [ ]:
with open("./MAL Anime Data/watching.json", "r") as file:
    watching = json.load(file)

with open("./MAL Anime Data/completed.json", "r") as file:
    completed = json.load(file)

In [ ]:
nodeColumns = ['id', 'title', 'genres', 'rating', 'media_type', 'studios']
listStatusColumns = ['status', 'start_date', 'finish_date']
other = ['color', 'label']

myMALDF_Anime = pd.DataFrame(columns=nodeColumns+listStatusColumns)

myMALDF_Anime = loadMALJSONIntoDF(watching, nodeColumns, listStatusColumns, myMALDF_Anime)
myMALDF_Anime = loadMALJSONIntoDF(completed, nodeColumns, listStatusColumns, myMALDF_Anime)
myMALDF_Anime['color'] = 'N/A'
myMALDF_Anime['label'] = 'N/A'
myMALDF_Anime

# Studio Challenge

This challenge is similar to the pre-existing studio challenge, but rather than forcing you to watch only one studio, it aims at exposing you to a lot of studios (40, to be exact) giving members the opportunity to find a studio they have never knew of before.

　This challenge is similar to the pre-existing studio challenge, but rather than forcing you to watch only one studio, it aims at exposing you to a lot of studios (40, to be exact) giving members the opportunity to find a studio they have never knew of before.

HOW-TO:
- Find the Studios List [here](https://myanimelist.net/anime/producer).
- You can use any studio, but your challenge must contain 40 different studios.
- Studios that are clearly "part" of another studio count as only 1 studio.
    - Example: Bandai includes > Bandai, Bandai Channel, Bandai Entertainment, etc.


Difficulty: complete a list of 40 different studios.
- If an anime has two or more studios, it can only count towards one for the challenge.
- This challenge can be completed thrice (x3). (added on Nov 05, 2019)

Previously completed series aren't allowed. Neither is re-watching.
- Series previously started, but not finished, may be used.
- Airing can be used, but you must complete the series, so you can only turn-in the challenge after the series has finished airing.

Only TV types, of any length are allowed.

Your challenge "Starting Date" is the day you post.
- Remember that it's required to add the start and end dates to any anime you use for our challenges.

You cannot use the same entry more than once across all difficulties.

To make this easier on the inspectors, you MUST list the studios in alphabetical order. Otherwise it's an instant invalid.

[Challenge Prompt](https://myanimelist.net/forum/?topicid=1504823)

[My Challenge Form](https://myanimelist.net/forum/?topicid=1504823&show=400#msg60311165)

In [ ]:
studioChallengeDate = '2020-07-25'
notEligibleMediaTypes = ['ona', 'movie', 'special', 'music', 'ova']

studiosChallengeDF = myMALDF_Anime.loc[:, ['id', 'title', 'status', 'media_type', 'studios', 'finish_date', 'color', 'label']]

indexValues = set()

indexValues.update(studiosChallengeDF.index[studiosChallengeDF['finish_date'] == 'N/A'].intersection(studiosChallengeDF.index[studiosChallengeDF['status'] == 'completed']))
indexValues.update(studiosChallengeDF.index[studiosChallengeDF['media_type'].isin(notEligibleMediaTypes)])
indexValues.update(studiosChallengeDF.index[studiosChallengeDF['finish_date'] < studioChallengeDate].intersection(studiosChallengeDF.index[studiosChallengeDF['status'] == 'completed']))
indexValues.update(studiosChallengeDF.index[~studiosChallengeDF['studios'].astype(bool)])

studiosChallengeDF.drop(indexValues, axis='index', inplace=True)

studiosChallengeDF.sort_values('finish_date', inplace=True, ignore_index=True)

studiosUsed = []
indexValues = []

for rowIndex in range(studiosChallengeDF.shape[0]):
    res = list(set(studiosChallengeDF.iloc[rowIndex]['studios']).difference(studiosUsed))

    if res:
        studiosUsed.append(res[0])
        studiosChallengeDF.at[rowIndex, 'studios'] = res[0]
        studiosChallengeDF.at[rowIndex, 'color'] = 'green' if studiosChallengeDF.at[rowIndex, 'status'] == 'completed' else 'orange'
        studiosChallengeDF.at[rowIndex, 'label'] = 'o' if studiosChallengeDF.at[rowIndex, 'status'] == 'completed' else 'x'
    else:
        indexValues.append(rowIndex)

studiosChallengeDF.drop(index=indexValues, columns=['media_type', 'finish_date'], inplace=True)
studiosChallengeDF.sort_values('studios', inplace=True, ignore_index=True)

studiosChallengeDF